In [9]:
import numpy as np
import cv2

In [10]:
def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

In [11]:
def preprocessing_harris(image):
    #1-Resizing image
    scale_percent = 30
    # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    shad_image = shadow_remove(resized_image)
    #3-Gray-Scale
    gray = cv2.cvtColor(shad_image, cv2.COLOR_BGR2GRAY)
    #2-Smoothing
    blured_image=cv2.GaussianBlur(gray,(5,5),cv2.BORDER_DEFAULT)
    #4 thresholding
    thresh = cv2.threshold(blured_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    #Remove dotted lines
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < cv2.contourArea(cnts[len(cnts)-1]):
            cv2.drawContours(thresh, [c], -1, (0,0,0), -1)
    # Fill contours
    close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = 255 - cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, close_kernel, iterations=6)
    # Smooth contours
    close = 255 - close
    return thresh,resized_image
# grayscale color space
#     gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
#     gray = np.float32(gray)
#     return gray,resized_image

In [12]:
# image = cv2.imread('E:/2nd term 3rd year/Neural Network/Project/men/'+str(2)+'/'+str(2)+'_men ('+str(27)+').JPG', cv2.IMREAD_UNCHANGED)
# img,r=preprocessing_harris(image)
# cv2.imshow('sample image',img)

# cv2.waitKey(0) # waits until a key is pressed
# cv2.destroyAllWindows() # destroys the window showing image